In [1]:
import cv2
import numpy as np

face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(face_cascade_path)

if face_cascade.empty():
    raise IOError("Failed to load Haar cascade file.")

def detect_mask(frame, face_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60), flags=cv2.CASCADE_SCALE_IMAGE)
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        mask_region = face[int(h/3):h, 0:w] 
        
        hsv = cv2.cvtColor(mask_region, cv2.COLOR_BGR2HSV)
        saturation = hsv[:, :, 1]
        
        brightness = cv2.mean(hsv[:, :, 2])[0]
        sat_threshold = max(50, brightness * 0.5) 
        
        avg_saturation = np.mean(saturation)
        
        lower_skin = np.array([0, 48, 80], dtype=np.uint8)
        upper_skin = np.array([20, 255, 255], dtype=np.uint8)
        skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)
        skin_ratio = cv2.countNonZero(skin_mask) / (w * int(h/2))
        
        if avg_saturation < sat_threshold and skin_ratio < 0.5:
            label = "Mask"
            color = (0, 255, 0)  # Green for mask
        else:
            label = "No Mask"
            color = (0, 0, 255)  # Red for no mask
        
   
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    return frame

print("[INFO] Starting video stream...")
vs = cv2.VideoCapture(0)

while True:
    ret, frame = vs.read()
    
    if not ret:
        break

    frame = detect_mask(frame, face_cascade)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.release()


[INFO] Starting video stream...
